In [1]:
import pandas as pd
import numpy as np
import datetime
from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import requests
from bs4 import BeautifulSoup
import time
import re
from urllib.request import urlopen
from itertools import combinations, permutations
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_rows', 200)

In [3]:
race_array = ["20220505","20220905","20220303","20220605","20220906","20220706","20230601","20230701","20231001","20230501"]

In [4]:
race_id_list = []
for race in race_array:
  for day in range(1, 10, 1):
      for r in range(1, 13, 1):
          race_id = race + str(day).zfill(2) + str(r).zfill(2)
          race_id_list.append(race_id)

In [5]:
len(race_id_list)

1080

In [6]:
def scrape(race_id_list):
  """
  レース結果データをスクレイピングする関数
  Parameters:
  ----------
  race_id_list : list
      レースIDのリスト
  Returns:
  ----------
  race_results_df : pandas.DataFrame
      全レース結果データをまとめてDataFrame型にしたもの
  """
  #race_idをkeyにしてDataFrame型を格納
  race_results = {}
  for race_id in tqdm(race_id_list):
      time.sleep(1)
      try:
          url = "https://db.netkeiba.com/race/" + race_id
          #メインとなるテーブルデータを取得
          df = pd.read_html(url)[0]
          html = requests.get(url)
          html.encoding = "EUC-JP"
          soup = BeautifulSoup(html.text, "html.parser")
          #天候、レースの種類、コースの長さ、馬場の状態、日付をスクレイピング
          texts = (
              soup.find("div", attrs={"class": "data_intro"}).find_all("p")[0].text
              + soup.find("div", attrs={"class": "data_intro"}).find_all("p")[1].text
          )
          info = re.findall(r'\w+', texts)
          for text in info:
              if text in ["芝", "ダート"]:
                  df["race_type"] = [text] * len(df)
              if "障" in text:
                  df["race_type"] = ["障害"] * len(df)
              if "m" in text:
                  df["course_len"] = [int(re.findall(r"\d+", text)[-1])] * len(df)
              if text in ["良", "稍重", "重", "不良"]:
                  df["ground_state"] = [text] * len(df)
              if text in ["曇", "晴", "雨", "小雨", "小雪", "雪"]:
                  df["weather"] = [text] * len(df)
              if "年" in text:
                  df["date"] = [text] * len(df)
          #馬ID、騎手IDをスクレイピング
          horse_id_list = []
          horse_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
              "a", attrs={"href": re.compile("^/horse")}
          )
          for a in horse_a_list:
              horse_id = re.findall(r"\d+", a["href"])
              horse_id_list.append(horse_id[0])
          jockey_id_list = []
          jockey_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
              "a", attrs={"href": re.compile("^/jockey")}
          )
          for a in jockey_a_list:
              jockey_id = re.findall(r"\d+", a["href"])
              jockey_id_list.append(jockey_id[0])
          df["horse_id"] = horse_id_list
          df["jockey_id"] = jockey_id_list
          #インデックスをrace_idにする
          df.index = [race_id] * len(df)
          race_results[race_id] = df
      #存在しないrace_idを飛ばす
      except IndexError:
          continue
      except AttributeError: #存在しないrace_idでAttributeErrorになるページもあるので追加
          continue
      #wifiの接続が切れた時などでも途中までのデータを返せるようにする
      except Exception as e:
          print(e)
          break
      #Jupyterで停止ボタンを押した時の対処
      except:
          break
  #pd.DataFrame型にして一つのデータにまとめる
  race_results_df = pd.concat([race_results[key] for key in race_results])
  return race_results_df

In [7]:
race_results_df = scrape(race_id_list)

  0%|          | 0/1080 [00:00<?, ?it/s]

In [26]:
#馬の過去成績データを処理するクラス
class HorseResults:
    @staticmethod
    def scrape(horse_id_list):
        """
        馬の過去成績データをスクレイピングする関数

        Parameters:
        ----------
        horse_id_list : list
            馬IDのリスト

        Returns:
        ----------
        horse_results_df : pandas.DataFrame
            全馬の過去成績データをまとめてDataFrame型にしたもの
        """

        #horse_idをkeyにしてDataFrame型を格納
        horse_results = {}
        for horse_id in tqdm(horse_id_list):
            time.sleep(1)
            try:
                url = 'https://db.netkeiba.com/horse/' + horse_id
                df = pd.read_html(url)[3]
                #受賞歴がある馬の場合、3番目に受賞歴テーブルが来るため、4番目のデータを取得する
                if df.columns[0]=='受賞歴':
                    df = pd.read_html(url)[4]
                df.index = [horse_id] * len(df)
                horse_results[horse_id] = df
            except IndexError:
                continue
            except Exception as e:
                print(e)
                break
            except:
                break

        #pd.DataFrame型にして一つのデータにまとめる        
        horse_results_df = pd.concat([horse_results[key] for key in horse_results])

        return horse_results_df

In [33]:
race_results_df

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,...,調教師,course_len,weather,race_type,ground_state,date,horse_id,jockey_id,year,month
202205050101,1,1,2,グローリーカフェ,牝2,54.0,田辺裕信,1:20.2,NaN,2.9,...,[東] 中舘英二,1300,曇,ダート,良,2022-11-05,2020106928,01075,2022,11
202205050101,2,4,8,アヴェッリーノ,牡2,55.0,津村明秀,1:20.7,3,70.4,...,[東] 的場均,1300,曇,ダート,良,2022-11-05,2020103747,01092,2022,11
202205050101,3,2,3,マラマプア,牝2,53.0,永野猛蔵,1:20.7,ハナ,8.6,...,[東] 伊藤圭三,1300,曇,ダート,良,2022-11-05,2020103454,01188,2022,11
202205050101,4,5,9,ヨロ,牡2,55.0,杉原誠人,1:20.8,クビ,44.2,...,[東] 大和田成,1300,曇,ダート,良,2022-11-05,2020106982,01135,2022,11
202205050101,5,2,4,マテリアルガール,牝2,54.0,内田博幸,1:20.9,3/4,9.7,...,[東] 西田雄一,1300,曇,ダート,良,2022-11-05,2020100787,00422,2022,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202310010412,12,4,8,パープルグローリー,牝4,51.0,永島まな,1:49.9,1/2,69.3,...,[西] 高橋康之,1800,小雨,芝,稍重,2023-01-22,2019101607,01187,2023,1
202310010412,13,7,13,スウィートスカー,牝4,55.0,藤岡康太,1:50.1,1.1/2,3.6,...,[西] 鮫島一歩,1800,小雨,芝,稍重,2023-01-22,2019106251,01116,2023,1
202310010412,14,6,12,ナックフローラ,牝4,55.0,勝浦正樹,1:50.2,クビ,31.3,...,[東] 杉浦宏昭,1800,小雨,芝,稍重,2023-01-22,2019104638,01025,2023,1
202310010412,15,2,3,ヘクトパスカル,牝4,55.0,菱田裕二,1:50.9,4,17.2,...,[西] 橋口慎介,1800,小雨,芝,稍重,2023-01-22,2019106048,01144,2023,1


In [31]:
race_results_df.to_pickle("results_2022_11to_2023_01.pickle")

In [32]:
horse_results.to_pickle("horse_results2022_11to_2023_01.pickle")

In [ ]:
#血統データを処理するクラス
class Peds:
    @staticmethod
    def scrape(horse_id_list):
        """
        血統データをスクレイピングする関数

        Parameters:
        ----------
        horse_id_list : list
            馬IDのリスト

        Returns:
        ----------
        peds_df : pandas.DataFrame
            全血統データをまとめてDataFrame型にしたもの
        """

        peds_dict = {}
        for horse_id in tqdm(horse_id_list):
            time.sleep(1)
            try:
                url = "https://db.netkeiba.com/horse/ped/" + horse_id
                df = pd.read_html(url)[0]

                #重複を削除して1列のSeries型データに直す
                generations = {}
                for i in reversed(range(5)):
                    generations[i] = df[i]
                    df.drop([i], axis=1, inplace=True)
                    df = df.drop_duplicates()
                ped = pd.concat([generations[i] for i in range(5)]).rename(horse_id)

                peds_dict[horse_id] = ped.reset_index(drop=True)
            except IndexError:
                continue
            except Exception as e:
                print(e)
                break
            except:
                break

        #列名をpeds_0, ..., peds_61にする
        peds_df = pd.concat([peds_dict[key] for key in peds_dict], axis=1).T.add_prefix('peds_')

        return peds_df